In [27]:
import pandas as pd

In [72]:
df = pd.read_csv('C:/Users/prate/Downloads/data1.csv')
df1 = df["sentence"]

df.head()

,sentence,sentiment
0,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,negative
1,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,negative
2,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,negative
3,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,negative
4,कश्मीर में हो रहा है जल जिहाद ।,negative


In [63]:
# Import and install the necessary libraries
from numba import cuda
from itertools import islice
import time
from tqdm import tqdm

# Install transformers
!pip install -q transformers

# Install the simpletransformers
!pip install -q simpletransformers

from simpletransformers.classification import ClassificationModel

In [65]:
model_args= {
            "num_train_epochs": 15,
            "learning_rate": 1e-5,
            "max_seq_length": 512,
             "silent": True
            }

model = ClassificationModel(
    "xlmroberta",
    "TajaKuzman/xlm-roberta-base-multilingual-text-genres",
    use_cuda=False,
    args=model_args
    
)

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/477 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/280 [00:00<?, ?B/s]

C:\Users\prate\anaconda3\lib\site-packages\simpletransformers\classification\classification_model.py:486: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(


In [66]:
prediction, logit_output = model.predict(["How to create a good text classification model? First step is to prepare good data. Make sure not to skip the exploratory data analysis. Pre-process the text if necessary for the task. The next step is to perform hyperparameter search to find the optimum hyperparameters. After fine-tuning the model, you should look into the predictions and analyze the model's performance. You might want to perform the post-processing of data as well and keep only reliable predictions."])

prediction_label = [model.config.id2label[i] for i in prediction]

print(prediction_label)

['Instruction']


In [67]:
# For genre prediction on a dataset: predict labels to the texts in batches (to make the prediction faster).

def predict(dataframe, dataframe_column, final_file):
  """
    The function takes the dataframe with text in column dataframe_column, creates batches of 8,
    and applies genre predictions on batches, for faster prediction.
    It saves the file with text and predictions with the final_file name.

    Args:
      - dataframe (pandas Dataframe): specify the dataframe
      - dataframe_column (str): specify which column in the dataframe has texts to which you want to predict genres, e.g. ("docs")
      - final_file: the name of the final file with predictions

  """
    # Split the dataframe into batches
    # Create batches of text
  def chunk(arr_range, arr_size):
      arr_range = iter(arr_range)
      return iter(lambda: tuple(islice(arr_range, arr_size)), ())

  batches_list = list(chunk(dataframe[dataframe_column], 8))

  batches_list_new = []

  for i in batches_list:
      batches_list_new.append(list(i))

  print("The dataset is split into {} batches of {} texts.".format(len(batches_list_new),len(batches_list_new[0])))

  y_pred = []

  print("Prediction started.")
  start_time = time.time()

  for i in tqdm(batches_list_new):
      output = model.predict(i)
      current_y_pred = [model.config.id2label[i] for i in output[0]]

      for i in current_y_pred:
          y_pred.append(i)

  prediction_time = round((time.time() - start_time)/60,2)

  print("\n\nPrediction completed. It took {} minutes for {} instances - {} minutes per one instance.".format(prediction_time, dataframe.shape[0], prediction_time/dataframe.shape[0]))

  dataframe["X-GENRE"] = y_pred

  # Save the new dataframe which contains the y_pred values as well
  dataframe.to_csv("{}".format(final_file), sep="\t")

  return dataframe

In [74]:
test_pred_df = predict (df.head(), "sentence", "Hindi-test-sample-predicted.csv")

The dataset is split into 1 batches of 5 texts.
Prediction started.


100%|██████████| 1/1 [07:35<00:00, 455.82s/it]



Prediction completed. It took 7.6 minutes for 5 instances - 1.52 minutes per one instance.



C:\Users\prate\AppData\Local\Temp\ipykernel_6092\946894499.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["X-GENRE"] = y_pred


In [75]:
test_pred_df.head()

,sentence,sentiment,X-GENRE
0,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,negative,Information/Explanation
1,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,negative,Prose/Lyrical
2,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,negative,News
3,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,negative,Other
4,कश्मीर में हो रहा है जल जिहाद ।,negative,News
